In [ ]:
# Step 1: Load Yahoo stock data (example: 'AAPL' stock)
ticker = 'AAPL'
df = yf.download(ticker, start="2010-01-01", end="2023-01-01")

# Step 2: Feature selection (we select Open, High, Low, Close, Volume, and Adj Close)
data = df[['Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']].values

# Step 3: Standardize the data
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

# Step 4: Prepare sequences (10 time steps, 6 variables)
def create_sequences(data, time_steps=10):
    sequences = []
    labels = []
    for i in range(len(data) - time_steps):
        sequences.append(data[i:i + time_steps])
        labels.append(data[i + time_steps, 3])  # Using 'Close' as the target label
    return np.array(sequences), np.array(labels)

time_steps = 10
X, y = create_sequences(data_scaled, time_steps)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Reshape X for Conv1D: (samples, time steps, variables)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2]))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], X_test.shape[2]))

# Step 5: Define the model architecture function (for GridSearchCV)
def create_model(filters=32, lstm_units=64, learning_rate=0.001):
    model = Sequential()
    model.add(Conv1D(filters=filters, kernel_size=1, activation=swish, input_shape=(time_steps, 6)))
    model.add(LSTM(lstm_units, activation=swish, return_sequences=False))
    model.add(Dense(1))

    # Compile the model
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model

# Step 6: Wrap Keras model with KerasRegressor for GridSearchCV
model = KerasRegressor(build_fn=create_model, verbose=0)

# Step 7: Define hyperparameters to tune
param_grid = {
    'filters': [16, 32, 64],  # Filters for Conv1D layer
    'lstm_units': [32, 64, 128],  # Hidden units for LSTM layer
    'learning_rate': [0.001, 0.01, 0.0001],  # Learning rates
    'batch_size': [32, 64],  # Batch size
    'epochs': [20, 50]  # Number of epochs
}

# Step 8: Define checkpoint callback to save the best model during training
checkpoint = ModelCheckpoint('best_model_during_training.h5', monitor='val_loss', save_best_only=True, mode='min', verbose=1)

# Step 9: Perform Grid Search for hyperparameter optimization
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3)

# Fit the model using grid search
grid_result = grid.fit(X_train, y_train, validation_data=(X_test, y_test), callbacks=[checkpoint])

# Step 10: Print best parameters and score
print(f"Best Parameters: {grid_result.best_params_}")
print(f"Best Score (MSE): {grid_result.best_score_}")

# Step 11: Save the final best model from GridSearchCV
best_model = grid_result.best_estimator_.model
best_model.save('best_final_model.h5')

# Step 12: Load the best model and evaluate on the test set
final_model = tf.keras.models.load_model('best_final_model.h5')

# Make predictions and destandardize
predictions = final_model.predict(X_test)
predictions_destandardized = scaler.inverse_transform(np.hstack((np.zeros((predictions.shape[0], 5)), predictions)))
predicted_close_prices = predictions_destandardized[:, 5]

# Evaluate the best model
test_loss = mean_squared_error(y_test, predicted_close_prices)
print(f"Test MSE: {test_loss}")


In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, LSTM, Dense
from tensorflow.keras.activations import swish
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
#Load Yahoo stock data (example: 'AAPL' stock)
ticker = 'AAPL'
df = yf.download(ticker, start="2010-01-01", end="2023-01-01")

[*********************100%***********************]  1 of 1 completed


In [3]:
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,7.622500,7.660714,7.585000,7.643214,6.454505,493729600
2010-01-05,7.664286,7.699643,7.616071,7.656429,6.465666,601904800
2010-01-06,7.656429,7.686786,7.526786,7.534643,6.362820,552160000
2010-01-07,7.562500,7.571429,7.466071,7.520714,6.351057,477131200
2010-01-08,7.510714,7.571429,7.466429,7.570714,6.393280,447610800
...,...,...,...,...,...,...
2022-12-23,130.919998,132.419998,129.639999,131.860001,130.631363,63814900
2022-12-27,131.380005,131.410004,128.720001,130.029999,128.818405,69007800
2022-12-28,129.669998,131.029999,125.870003,126.040001,124.865585,85438400


In [4]:
#Feature selection (we select Open, High, Low, Close, Volume, and Adj Close)
data = df[['Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']].values

In [5]:
#Standardizing the data
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

In [6]:
def create_sequences(data, time_steps=10):
    sequences = []
    labels = []
    for i in range(len(data) - time_steps):
        sequences.append(data[i:i + time_steps])
        labels.append(data[i + time_steps, 3])  # Using 'Close' as the target label
    return np.array(sequences), np.array(labels)

In [7]:
time_steps = 10
X, y = create_sequences(data_scaled, time_steps)

In [8]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Reshaping X for Conv1D: (samples, time steps, variables)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2]))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], X_test.shape[2]))


In [9]:
# Model architecture
def create_model(filters=32, lstm_units=64, learning_rate=0.001):
    model = Sequential()
    model.add(Conv1D(filters=filters, kernel_size=1, activation=swish, input_shape=(time_steps, 6)))
    model.add(LSTM(lstm_units, activation=swish, return_sequences=False))
    model.add(Dense(1))

    # Compile the model
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model

In [11]:
#Gridsearch for hyperparameter optimization
filter_options = [16, 32, 64]  # Filters for Conv1D layer
lstm_units_options = [32, 64, 128]  # Hidden units for LSTM layer
learning_rate_options = [0.001, 0.01, 0.0001]  # Learning rates
batch_size_options = [32, 64]  # Batch size
epochs_options = [20, 50]  # Number of epochs

In [12]:
# To store the results of each run
best_loss = float('inf')
best_params = None
best_model = None

In [13]:
#Looping over hyperparameter combinations
for filters in filter_options:
    for lstm_units in lstm_units_options:
        for learning_rate in learning_rate_options:
            for batch_size in batch_size_options:
                for epochs in epochs_options:
                    print(f"Training with filters={filters}, lstm_units={lstm_units}, learning_rate={learning_rate}, batch_size={batch_size}, epochs={epochs}")

                    # Create model with current hyperparameters
                    model = create_model(filters=filters, lstm_units=lstm_units, learning_rate=learning_rate)

                    # Define checkpoint callback to save the best model during training
                    checkpoint = ModelCheckpoint('temp_model.h5', monitor='val_loss', save_best_only=True, mode='min', verbose=1)

                    # Train the model
                    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), callbacks=[checkpoint], verbose=1)

                    # Load the best model
                    model.load_weights('temp_model.h5')

                    # Evaluate on test set
                    predictions = model.predict(X_test)
                    predictions_destandardized = scaler.inverse_transform(np.hstack((np.zeros((predictions.shape[0], 5)), predictions)))
                    predicted_close_prices = predictions_destandardized[:, 5]

                    # Calculate MSE on test set
                    test_loss = mean_squared_error(y_test, predicted_close_prices)
                    print(f"Test MSE: {test_loss}")

                    # Check if this is the best model so far
                    if test_loss < best_loss:
                        best_loss = test_loss
                        best_params = (filters, lstm_units, learning_rate, batch_size, epochs)
                        best_model = model

Training with filters=16, lstm_units=32, learning_rate=0.001, batch_size=32, epochs=20
Epoch 1/20
78/82 [===========================>..] - ETA: 0s - loss: 0.0884
Epoch 1: val_loss improved from inf to 2.15954, saving model to temp_model.h5
82/82 [==============================] - 7s 18ms/step - loss: 0.0851 - val_loss: 2.1595
Epoch 2/20
14/82 [====>.........................] - ETA: 0s - loss: 0.0089

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


80/82 [============================>.] - ETA: 0s - loss: 0.0056
Epoch 2: val_loss did not improve from 2.15954
82/82 [==============================] - 1s 10ms/step - loss: 0.0055 - val_loss: 784.5220
Epoch 3/20
82/82 [==============================] - ETA: 0s - loss: 0.0011
Epoch 3: val_loss did not improve from 2.15954
82/82 [==============================] - 1s 11ms/step - loss: 0.0011 - val_loss: 93.7896
Epoch 4/20
80/82 [============================>.] - ETA: 0s - loss: 7.9794e-04
Epoch 4: val_loss did not improve from 2.15954
82/82 [==============================] - 1s 10ms/step - loss: 7.9120e-04 - val_loss: 8.0534
Epoch 5/20
81/82 [============================>.] - ETA: 0s - loss: 6.3672e-04
Epoch 5: val_loss improved from 2.15954 to 1.99323, saving model to temp_model.h5
82/82 [==============================] - 1s 10ms/step - loss: 6.3311e-04 - val_loss: 1.9932
Epoch 6/20
79/82 [===========================>..] - ETA: 0s - loss: 5.2859e-04
Epoch 6: val_loss improved from 1.9932

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


82/82 [==============================] - ETA: 0s - loss: 0.0015
Epoch 2: val_loss did not improve from 0.33841
82/82 [==============================] - 1s 15ms/step - loss: 0.0015 - val_loss: 41.9107
Epoch 3/50
79/82 [===========================>..] - ETA: 0s - loss: 9.5435e-04
Epoch 3: val_loss did not improve from 0.33841
82/82 [==============================] - 1s 15ms/step - loss: 9.7476e-04 - val_loss: 12.8351
Epoch 4/50
78/82 [===========================>..] - ETA: 0s - loss: 7.9840e-04
Epoch 4: val_loss did not improve from 0.33841
82/82 [==============================] - 1s 15ms/step - loss: 7.9584e-04 - val_loss: 4.8909
Epoch 5/50
78/82 [===========================>..] - ETA: 0s - loss: 6.7769e-04
Epoch 5: val_loss did not improve from 0.33841
82/82 [==============================] - 1s 11ms/step - loss: 6.7437e-04 - val_loss: 2.0994
Epoch 6/50
78/82 [===========================>..] - ETA: 0s - loss: 5.9053e-04
Epoch 6: val_loss did not improve from 0.33841
82/82 [============

...

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


38/41 [==========================>...] - ETA: 0s - loss: 0.0062
Epoch 2: val_loss improved from 1.50839 to 0.30931, saving model to temp_model.h5
41/41 [==============================] - 1s 16ms/step - loss: 0.0060 - val_loss: 0.3093
Epoch 3/50
39/41 [===========================>..] - ETA: 0s - loss: 0.0016
Epoch 3: val_loss did not improve from 0.30931
41/41 [==============================] - 1s 15ms/step - loss: 0.0015 - val_loss: 1257.0334
Epoch 4/50
39/41 [===========================>..] - ETA: 0s - loss: 7.2928e-04
Epoch 4: val_loss did not improve from 0.30931
41/41 [==============================] - 1s 14ms/step - loss: 7.2399e-04 - val_loss: 260.3767
Epoch 5/50
41/41 [==============================] - ETA: 0s - loss: 6.1014e-04
Epoch 5: val_loss did not improve from 0.30931
41/41 [==============================] - 1s 22ms/step - loss: 6.1014e-04 - val_loss: 67.4868
Epoch 6/50
38/41 [==========================>...] - ETA: 0s - loss: 5.2216e-04
Epoch 6: val_loss did not improve f

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


80/82 [============================>.] - ETA: 0s - loss: 3.7761e-04
Epoch 2: val_loss did not improve from 0.14179
82/82 [==============================] - 1s 12ms/step - loss: 3.8357e-04 - val_loss: 1.6735
Epoch 3/20
82/82 [==============================] - ETA: 0s - loss: 3.4988e-04
Epoch 3: val_loss did not improve from 0.14179
82/82 [==============================] - 1s 17ms/step - loss: 3.4988e-04 - val_loss: 4.7349
Epoch 4/20
81/82 [============================>.] - ETA: 0s - loss: 3.7235e-04

KeyboardInterrupt: 

In [14]:
# Saving the best model
print(f"Best Hyperparameters: filters={best_params[0]}, lstm_units={best_params[1]}, learning_rate={best_params[2]}, batch_size={best_params[3]}, epochs={best_params[4]}")
best_model.save('best_final_model.h5')

Best Hyperparameters: filters=16, lstm_units=32, learning_rate=0.0001, batch_size=64, epochs=20


In [15]:
# Evaluating the best model on the test set again
final_predictions = best_model.predict(X_test)
final_predictions_destandardized = scaler.inverse_transform(np.hstack((np.zeros((final_predictions.shape[0], 5)), final_predictions)))
predicted_close_prices_final = final_predictions_destandardized[:, 5]


21/21 [==============================] - 0s 3ms/step


In [16]:
# Evaluating the best model
final_test_loss = mean_squared_error(y_test, predicted_close_prices_final)
print(f"Final Test MSE: {final_test_loss}")

Final Test MSE: 4849.877622904099
